In [1]:
import psycopg2 as pg
import sqlalchemy
import petl as etl
from datetime import datetime
import pytz

In [16]:
from petl import datetimeparser
isodatetime = datetimeparser('%Y-%m-%d %H:%M:%S')

In [118]:
conn = pg.connect("host=172.20.0.2 dbname=testcase user=airflow password=airflow")

In [119]:
delta_info = etl.fromdb(conn, 'SELECT count(1)+1 count_rec,max(end_ts) last_e_ts FROM olist.olist_delta')
delta_info

count_rec,last_e_ts
1,None


In [95]:
now = datetime.now()
now_str = now.strftime("%Y-%d-%m %H:%M:%S")
lv_last_e_ts = datetime(2000,1,1,0,0,0) if delta_info['last_e_ts'][0] == None else delta_info['last_e_ts'][0] 
lv_delta_id = delta_info['count_rec'][0]
lv_last_e_ts
lv_delta_id

1

In [73]:
table1 = etl.fromcsv(source = '/home/jovyan/work/test1.csv',delimiter = ';')
table1 = etl.convert(table1,'id',int)
table1

id,data,modi_date
1,sdsdsd,2012-01-07 00:00:00
2,sdsdjfhsd,2012-01-07 00:01:00
3,x,2012-01-07 00:00:00
4,d,2012-01-08 00:00:00
5,sdd,2012-01-07 00:00:00


In [74]:
table1 = etl.convert(table1,{"modi_date": lambda d: isodatetime(d)})
table1

id,data,modi_date
1,sdsdsd,2012-01-07 00:00:00
2,sdsdjfhsd,2012-01-07 00:01:00
3,x,2012-01-07 00:00:00
4,d,2012-01-08 00:00:00
5,sdd,2012-01-07 00:00:00


In [120]:
testdate = datetime(2012, 1, 7, 0, 1,0)
table2 =  etl.select(table1,lambda rec: rec.modi_date >= lv_last_e_ts and rec.modi_date < now)
table3 = etl.addfield(table2,'load_date',now)
table3.nrows()

9

In [121]:
etl.todb(table =  table3 ,dbo = conn, tablename = "test_csvpetl_2", schema = 'olist', drop = True)

In [122]:
delta_table_new = etl.setheader([['delta_id','start_ts','end_ts','delta_object','ts'],
                                 [lv_delta_id,lv_last_e_ts,now,None,now]],
                                ['delta_id','start_ts','end_ts','delta_object','ts'])
delta_table_new

delta_id,start_ts,end_ts,delta_object,ts
1,2000-01-01 00:00:00,2021-08-06 06:14:27.349629,None,2021-08-06 06:14:27.349629


In [123]:
etl.todb(table =  delta_table_new ,dbo = conn, tablename = "olist_delta", schema = 'olist')

In [124]:
conn.close()